# Imports

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

import os

import openpyxl
from os import listdir
from os.path import isfile, join

from sklearn.metrics import accuracy_score

dataDir = './archive/imagesMainWork'
anotation_path = './archive/annotations'

# Function to Detect Red Circles

In [2]:
# CLAHE

def clahe(testImg):
    testImg = cv2.cvtColor(testImg, cv2.COLOR_RGB2HSV)

    h, s, v = testImg[:,:,0], testImg[:,:,1], testImg[:,:,2]

    clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (8, 8))

    v = clahe.apply(v)

    testImg = np.dstack((h, s, v))

    testImg = cv2.cvtColor(testImg, cv2.COLOR_HSV2RGB)
    
    return testImg

# REMOVE SHADOWS

def removeShadows(testImg):
    
    rgb_planes = cv2.split(testImg)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)

    testImg = cv2.merge(result_norm_planes)
    
    return testImg

#

def redMask(testImg):
    testImg = cv2.cvtColor(testImg, cv2.COLOR_RGB2HSV)

    # lower mask (0-10)
    lower_red = np.array([0,50,50])
    upper_red = np.array([10,255,255])
    mask0 = cv2.inRange(testImg, lower_red, upper_red)

    # upper mask (170-180)
    lower_red = np.array([170,50,50])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(testImg, lower_red, upper_red)

    # join my masks
    mask = mask0+mask1

    # set my output img to zero everywhere except my mask
    testImgRed = ogImg.copy()
    
    testImgRed = cv2.cvtColor(testImgRed, cv2.COLOR_RGB2HSV)
    
    testImgRed[np.where(mask==0)] = 0
    
    testImgRed = cv2.cvtColor(testImgRed, cv2.COLOR_HSV2RGB)
    
    testImg = cv2.cvtColor(testImg, cv2.COLOR_HSV2RGB)
    
    return testImgRed


# CLOSING

def closing(testImgRed):
    testImgRed = cv2.GaussianBlur(testImgRed, (11,11), 3)
    testImgGray = cv2.cvtColor(testImgRed, cv2.COLOR_RGB2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    testImgGray = cv2.morphologyEx(testImgGray, cv2.MORPH_CLOSE, kernel)

    return testImgGray



def findCircles(testImgRed):
    # Use the Hough transform to detect circles in the image
    circles = cv2.HoughCircles(testImgRed, cv2.HOUGH_GRADIENT, 1, testImgRed.shape[0] / 8, param1=50, param2=45, minRadius=0, maxRadius=0)
    # If we have extracted a circle, draw an outline
    # We only need to detect one circle here, since there will only be one reference object
    return circles

def detectCircles(ogImage):
    finalImg = ogImg.copy()
    testImg = ogImg.copy()
            
    for i in range(3):
        testImg = ogImg.copy()
        if i == 0:
            noShadows = removeShadows(testImg)
            redOnly = redMask(noShadows)
            grayClosing = closing(redOnly)
            circles = findCircles(grayClosing)        
        if i == 1:
            claheImg = clahe(testImg)
            redOnly = redMask(claheImg)
            grayClosing = closing(redOnly)
            circles = findCircles(grayClosing)
        if i == 2:
            redOnly = redMask(testImg)
            grayClosing = closing(redOnly)
            circles = findCircles(grayClosing)
        if circles is not None:
            print(circles.shape[1])
            return circles.shape[1]
    return 0

# Function to detect blue squares - Pre Process

In [ ]:
def smooth(img): 

    # remove noise
    # Using a Median Filter
    img_smoothed = cv2.medianBlur(img, 5)

    return img_smoothed
    # return img

def contrastAdjust(img):

    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    h, s, v = img[:, :, 0], img[:, :, 1], img[:, :, 2]

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

    v = clahe.apply(v)

    img = np.dstack((h, s, v))

    img_clean = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
    
    return img_clean

def removeShadows(img):
    rgb_planes = cv2.split(img)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7, 7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 41)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(
            diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)

    img_clean = cv2.merge(result_norm_planes)

    return img_clean


# Function to detect blue squares - Angles

In [ ]:
# TODO: CHANGE THIS

def calcAngles(cnt_img, corners):
    angles = []
    
    # Calculate euclidean distance between each corner
    dist1 = int(math.sqrt(math.pow(corners[0] - corners[2], 2) +
                       math.pow(corners[1] - corners[3], 2)))
    dist2 = int(math.sqrt(math.pow(corners[2] - corners[4], 2) +
                       math.pow(corners[3] - corners[5], 2)))
    dist3 = int(math.sqrt(math.pow(corners[4] - corners[6], 2) +
                   math.pow(corners[5] - corners[7], 2)))
    dist4 = int(math.sqrt(math.pow(corners[6] - corners[0], 2) +
                   math.pow(corners[7] - corners[1], 2)))

    
    max_radius = min([dist1, dist2, dist3, dist4])

    for i in range(len(corners[:-1]), 2):
        blank_img = np.zeros((len(cnt_img), len(cnt_img[0])), np.uint8)
        cv2.circle(blank_img, (corners[i], corners[i + 1]),
                  max_radius // 2, (255, 255, 255))
        intersect_img = cv2.bitwise_and(cnt_img, blank_img)

        intersect_pts = np.where(intersect_img > 1)

        if(len(intersect_pts[0]) < 2 or len(intersect_pts[1]) < 2):
            angles.append(0)
            continue

        vector1 = (intersect_pts[1][0] - corners[i],
                   intersect_pts[0][0] - corners[i + 1])
        vector2 = (intersect_pts[1][1] - corners[i],
                   intersect_pts[0][1] - corners[i + 1])

        scalar_p = vector1[0] * vector2[0] + vector1[1] * vector2[1]
        norm1 = math.sqrt(math.pow(vector1[0], 2) + math.pow(vector1[1], 2))
        norm2 = math.sqrt(math.pow(vector2[0], 2) + math.pow(vector2[1], 2))

        angle = math.acos(scalar_p / (norm1 * norm2)) * 180 / math.pi
        angles.append(angle)

    return angles

# Function to detect blue squares - Feature Recognition

In [ ]:
def featureRecognition(img): 

    # Segmentation to recognize colour blue
    # Converts images from BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Bounds adjusted to identify shades of blue correspondent to traffic signs
    lower_blue = np.array([100, 100, 80])
    upper_blue = np.array([150, 255, 255])

    # Here we are defining range of bluecolor in HSV
    # This creates a mask of blue coloured
    # objects found in the frame.
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # The bitwise and of the frame and mask is done so
    # that only the blue coloured objects are highlighted
    # and stored in res
    res = cv2.bitwise_and(img, img, mask=mask)


    # Post-Segmentation smoothing 
    # res = smooth(res)  


    # Contours on Resulting Image
    gray_img = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    # Apply Canny
    canny = cv2.Canny(gray_img, 130, 255)

    # Find Contours
    # RETR_CCOMP: retrieves all of the contours and organizes them into a two-level hierarchy. At the top level, there are external boundaries of the components. 
    # At the second level, there are boundaries of the holes. If there is another contour inside a hole of a connected component, it is still put at the top level.
    cnts = cv2.findContours(
        canny, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]


    for c in range(len(cnts)):
        cnt_len = cv2.arcLength(cnts[c], True)

        # Small contours will appear in green; can be commented and code will still work
        if(cnt_len <= 70):
            cv2.drawContours(img, [cnts[c]], 0, (0, 255, 0), 3)
            continue 

        # Recognize most approximate CLOSED Poly-shape to contour to draw
        approx_poly = cv2.approxPolyDP(
            cnts[c], 0.03 * cv2.arcLength(cnts[c], True), True)

        # Rectangle Recognition through Distance and Angle Analysis
        ravel = approx_poly.ravel()
        n_sides = len(approx_poly)
        angle_cdt = False 

        # Nº of Sides is 4 = Square/Rectangle
        if(n_sides == 4): 
            angle_cdt = True 

            cnt_img = np.zeros((len(img), len(img[0])), np.uint8)
            angles = calcAngles(cv2.drawContours(cnt_img, [approx_poly], -1, (255), 1), ravel)

            for a in angles:
                if(abs(a - 90) > 10):
                    angle_cdt=False
                    break
            
        if(angle_cdt):
            img = cv2.drawContours(img, [approx_poly], -1, (0, 230, 255), 3)

In [ ]:
#SIFT & FLANN & RANSAC
def featureDetectionAndMatching(img, train_list):

    MIN_MATCH_COUNT = 5
    count = 0
    for img_path in train_list:

        img_train = cv2.imread(img_path)

        # Initiate SIFT detector
        sift = cv2.SIFT_create()

        
        # find the keypoints & compute descriptors
        kp, des = sift.detectAndCompute(img_train, None)
        kp2, des2 = sift.detectAndCompute(img, None)


        # create FLANN feature matcher
        FLANN_INDEX_KDTREE = 1

        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(des, des2, k=2)

        # store all the good matches as per Lowe's ratio test.
        good = []
        for m, n in matches:
            if m.distance < 0.7*n.distance:
                good.append(m)

        if len(good) >= MIN_MATCH_COUNT:
            src_pts = np.float32([kp[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            matchesMask = mask.ravel().tolist()
            h, w = img_train.shape[:2]
            pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]
                            ).reshape(-1, 1, 2)
            if not M is None:
                dst = cv2.perspectiveTransform(pts, M)
                img = cv2.polylines(img, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)
            else:   
                continue
        else:
            matchesMask = None
            continue

        count+=1

    return count


# Loop to analize the entire data set (download from kaggle)

In [ ]:
# TODO: meter mais imagens de limite de velocidade recomendado
train_set = ['./train-img/crossing.png', './train-img/crossing_1.png',
             './train-img/passage_priority.png', './train-img/dead_end.png', './train-img/roads_limits.png', './train-img/via_reservada.png', './train-img/via_reservada_end.png', './train-img/info.png', './train-img/information-lane-end.png', './train-img/information-lane-start.png', './train-img/information-lanes.png', './train-img/information-motorway_end.png', './train-img/information-motorway.png', './train-img/information-one-way-traffic_end.png', './train-img/information-one-way-traffic.png', './train-img/information-parking.png', './train-img/information-speed-recommended_end.png', './train-img/information-speed-recommended.png', './train-img/information-tunnel_end.png', './train-img/information-tunnel.png', './train-img/information-two-way-traffic.png']


onlyfiles = [f for f in listdir(dataDir) if isfile(join(dataDir, f))]
 
workbook = openpyxl.load_workbook("RoadSigns.xlsx")
 
sheet = workbook.active
# Start from the first cell.
# Rows and columns are zero indexed.
row_number = 2

print(len(onlyfiles))
# iterating through content list
for item in onlyfiles :
    
    
    ogImg = cv2.imread(os.path.join(dataDir, item))
    
    #img_clean = removeShadows(contrastAdjust(ogImg))
    
    #nrBlueSquares = featureDetectionAndMatching(img_clean, train_set)
    
    cv2.imshow("teste",ogImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    ogImgRGB = cv2.cvtColor(ogImg, cv2.COLOR_BGR2RGB)
    
    nrCircles = detectCircles(ogImgRGB)
    
    if nrCircles > 0:
        sheet.cell(row = row_number, column = 3, value = "speedlimit")
    else: 
        sheet.cell(row = row_number, column = 3, value = "other")
    #sheet.cell(row = row_number, column = 7, value = nrBlueSquares)
    

    row_number += 1

workbook.save("RoadSigns.xlsx")

535
1
1
1
1
1
1
1
1
1
1
1
1
2





# Calculate the accuracy

In [ ]:
# Red Circles Accuracy
actual = sheet['B']
detected = sheet['C']

def mapping(cell):
    return cell.value

actual = list(map(mapping, actual[1:]))
detected = list(map(mapping, detected[1:]))
accuracy = round(accuracy_score(actual, detected) * 100)

print("Accuracy for Red Circles: {}%".format(accuracy))

# # Blue Squares/Rectangles Accuracy
# actual = sheet['D']
# detected = sheet['G']

# def mapping(cell):
#     return cell.value

# actual = list(map(mapping, actual[1:]))
# detected = list(map(mapping, detected[1:]))

# accuracy = round(accuracy_score(actual, detected) * 100)

# print("Accuracy for Blue Squares/Rectangles: {}%".format(accuracy))

# # Stops Accuracy
# actual = sheet['B']
# detected = sheet['E']

# def mapping(cell):
#     return cell.value

# actual = list(map(mapping, actual[1:]))
# detected = list(map(mapping, detected[1:]))

# accuracy = round(accuracy_score(actual, detected) * 100)

# print("Accuracy for Stops: {}%".format(accuracy))



# Run to generate the initial file 
## Filtered dataset information processed - 85%

In [ ]:
# import xlsxwriter module
import xlsxwriter
import xml.dom.minidom


from os import listdir
from os.path import isfile, join
onlyfiles = [".".join(f.split(".")[:-1]) for f in listdir(dataDir) if isfile(join(dataDir, f))]
 
workbook = xlsxwriter.Workbook('RoadSigns.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'File Name')
worksheet.write('B1', 'Type of Sign')
worksheet.write('C1', 'Type of Sign Detected')
# Start from the first cell.
# Rows and columns are zero indexed.
row = 1
column = 0

 
# iterating through content list
for item in onlyfiles :
    typeOfSign = "";
    doc = xml.dom.minidom.parse(os.path.join(anotation_path,item + ".xml"));
    
    signsNames = doc.getElementsByTagName("name")
    
    for sign in signsNames:
     if sign.childNodes[0].nodeValue == "stop":
        typeOfSign = "stop";
     elif sign.childNodes[0].nodeValue == "speedlimit":
        typeOfSign = "speedlimit";
     elif sign.childNodes[0].nodeValue == "crosswalk":
        typeOfSign = "crosswalk";
    
    # write operation perform
    if typeOfSign == "speedlimit":
        worksheet.write(row, column, item)
        worksheet.write(row, 1, typeOfSign)
        # incrementing the value of row by one
        # with each iterations.
    else:
        worksheet.write(row, column, item)
        worksheet.write(row, 1, "other")
 
    row += 1
     
workbook.close()